In [2]:
from pathlib import Path

import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from sample.sentinel_downloader import SentinelDownloader, Season
from sample.sentinel_cities_downloader import SentinelCitiesDownloader
from sample.osm_road_data import OSMRoadData
from sample.osm_road_generator import OSMRoadGenerator

In [3]:
cities_geojson = Path("../data/us14_city_ids_and_bounds.geojson")
if not cities_geojson.exists():
    raise FileNotFoundError(cities_geojson)

cities_gdf = gpd.read_file(cities_geojson)

In [4]:
gdf_proj = cities_gdf.to_crs(3857)
gdf_proj["area"] = gdf_proj.area

In [5]:
sorted_by_area = gdf_proj.sort_values(by="area")
sorted_by_area[:3].to_crs(4326).to_file("../data/small_city_ids_and_bounds.geojson")

In [13]:
sorted_by_area[:3]

,asset_name,asset_identifier,iso3_country,geometry,area
13,Hartford,13,USA,"POLYGON ((-8094890.434 5132052.793, -8094784.7...",8.372409e+07
10,Pittsburgh,10,USA,"POLYGON ((-8910543.705 4924075.262, -8910866.1...",2.641764e+08
12,Orlando,12,USA,"MULTIPOLYGON (((-9054581.887 3307602.554, -905...",3.769789e+08


In [8]:
subset = sorted_by_area[sorted_by_area["asset_name"].isin(["Pittsburgh", "Cleveland", "Denver"])]

In [9]:
subset.to_file("../data/small_city_ids_and_bounds.geojson")

In [ ]:
s2_cities_downloader = SentinelCitiesDownloader(years=[2021], 
                                                seasons=[Season.Spring, Season.Summer, Season.Fall],)
s2_cities_downloader.download_all(parallel=False, debug=False)

In [ ]:
osm_road_generator = OSMRoadGenerator()
osm_road_generator.generate_roads_parallel()

In [ ]:
plot_city_id = 10
city_name = cities_gdf[cities_gdf["asset_identifier"] == plot_city_id]["asset_name"].iloc[0]

In [ ]:
with rasterio.open(f"data/sentinel2_images/{plot_city_id}/2021/summer.tif") as ds:
    visual_img = np.moveaxis(ds.read(), 0, -1)
with rasterio.open(f"data/osm_images/{plot_city_id}.tif") as ds:
    road_img = np.moveaxis(ds.read(), 0, -1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(visual_img)
axes[0].get_yaxis().set_visible(False)
axes[0].get_xaxis().set_visible(False)
axes[1].imshow(road_img)
axes[1].get_yaxis().set_visible(False)
axes[1].get_xaxis().set_visible(False)
plt.suptitle(city_name)
